In [1]:
import pandas as pd

# 1. 파일 불러오기
df = pd.read_csv('kbo_batting_stats_by_season_1982-2025.csv')

# 2. 팀명 통합 및 연도 필터링 (2020~2025)
df['Team'] = df['Team'].replace('SK', 'SSG')
years = range(2020, 2026)
df_filtered = df[df['Year'].isin(years)].copy()

# 3. 가중 평균 wRC+ 계산을 위한 보조 값 생성 (wRC+ * PA)
df_filtered['wRC_weighted'] = df_filtered['wRC+'] * df_filtered['PA']

# 4. 그룹화 및 집계 (팀별 -> 연도별)
# WAR, oWAR, PA, wRC_weighted는 합산합니다.
team_stats = df_filtered.groupby(['Team', 'Year']).agg({
    'WAR': 'sum',
    'oWAR': 'sum',
    'PA': 'sum',
    'wRC_weighted': 'sum'
}).reset_index()

# 5. 가중 평균 wRC+ 최종 계산 (합산된 weighted_wRC / 전체 PA)
team_stats['wRC+_avg'] = team_stats['wRC_weighted'] / team_stats['PA']

# 6. 보기 좋게 컬럼 정리 및 반올림
team_stats = team_stats[['Team', 'Year', 'WAR', 'oWAR', 'wRC+_avg']]
team_stats = team_stats.round(2)

# 7. 요청하신 팀 순서대로 정렬
team_order = ['LG', '한화', 'SSG', '삼성', 'NC', 'KT', '롯데', 'KIA', '두산', '키움']
team_stats['Team'] = pd.Categorical(team_stats['Team'], categories=team_order, ordered=True)
team_stats = team_stats.sort_values(['Team', 'Year'])

# 8. 구단별로 분리해서 출력 및 저장
print("=== 10개 구단별 연도별 타격 요약 데이터 ===")
for team in team_order:
    print(f"\n[{team} 라이온즈/랜더스 등 팀 기록]") # 팀별 출력
    display_df = team_stats[team_stats['Team'] == team]
    print(display_df.to_string(index=False))
    
    # 개별 파일로도 저장하고 싶다면 아래 주석 해제
    # display_df.to_csv(f'{team}_타격_요약_2020_2025.csv', index=False, encoding='utf-8-sig')

# 전체 통합 요약본 저장
team_stats.to_csv('KBO_팀별_연도별_타격요약_통합.csv', index=False, encoding='utf-8-sig')

=== 10개 구단별 연도별 타격 요약 데이터 ===

[LG 라이온즈/랜더스 등 팀 기록]
Team  Year   WAR  oWAR  wRC+_avg
  LG  2020 27.56 27.12    111.65
  LG  2021 22.81 21.54    101.32
  LG  2022 32.05 29.80    115.75
  LG  2023 33.67 32.99    118.56
  LG  2024 30.35 28.10    109.14
  LG  2025 36.33 34.92    118.67

[한화 라이온즈/랜더스 등 팀 기록]
Team  Year   WAR  oWAR  wRC+_avg
  한화  2020  2.02  3.27     75.87
  한화  2021 13.49 11.72     88.32
  한화  2022 12.60 12.48     88.06
  한화  2023 14.01 13.80     89.36
  한화  2024 13.05 14.73     93.59
  한화  2025 20.00 19.19     99.39

[SSG 라이온즈/랜더스 등 팀 기록]
Team  Year   WAR  oWAR  wRC+_avg
 SSG  2020  8.85  9.04     84.27
 SSG  2021 28.00 28.55    109.40
 SSG  2022 28.27 24.75    103.12
 SSG  2023 21.13 22.40    100.08
 SSG  2024 15.15 17.17     92.90
 SSG  2025 15.09 14.56     91.72

[삼성 라이온즈/랜더스 등 팀 기록]
Team  Year   WAR  oWAR  wRC+_avg
  삼성  2020 13.92 13.23     86.72
  삼성  2021 18.14 19.34     97.99
  삼성  2022 17.06 19.27     97.97
  삼성  2023 16.37 16.64     94.31
  삼성  2024 20.08 16.74 

In [2]:
import pandas as pd

# 1. 파일 불러오기
df = pd.read_csv('kbo_batting_stats_by_season_1982-2025.csv')

# 2. 팀명 통합 및 연도 필터링 (2020~2025)
df['Team'] = df['Team'].replace('SK', 'SSG')
years = range(2020, 2026)
df_filtered = df[df['Year'].isin(years)].copy()

# 3. 가중 평균 wRC+ 계산을 위한 보조 값 생성 (wRC+ * PA)
# wRC+가 NaN인 경우 0으로 처리하여 계산 오류 방지
df_filtered['wRC+'] = df_filtered['wRC+'].fillna(0)
df_filtered['wRC_weighted'] = df_filtered['wRC+'] * df_filtered['PA']

# 4. 그룹화 및 집계 (팀별 -> 연도별)
team_stats = df_filtered.groupby(['Team', 'Year']).agg({
    'WAR': 'sum',
    'oWAR': 'sum',
    'PA': 'sum',
    'wRC_weighted': 'sum'
}).reset_index()

# 5. 가중 평균 wRC+ 최종 계산
# 타석(PA)이 0인 경우를 대비해 나누기 전 체크
team_stats['wRC+_avg'] = team_stats.apply(
    lambda x: x['wRC_weighted'] / x['PA'] if x['PA'] > 0 else 0, axis=1
)

# 6. 보기 좋게 컬럼 정리 및 반올림
team_stats = team_stats[['Team', 'Year', 'WAR', 'oWAR', 'wRC+_avg']]
team_stats = team_stats.round(2)

# 7. 요청하신 팀 순서대로 정렬
team_order = ['LG', '한화', 'SSG', '삼성', 'NC', 'KT', '롯데', 'KIA', '두산', '키움']
team_stats['Team'] = pd.Categorical(team_stats['Team'], categories=team_order, ordered=True)
team_stats = team_stats.sort_values(['Team', 'Year'])

# 8. 구단별 분리 출력 및 저장
print("=== 10개 구단별 연도별 타격 요약 데이터 ===")
for team in team_order:
    # 기업 이름만 깔끔하게 출력하도록 수정
    print(f"\n[{team}]") 
    display_df = team_stats[team_stats['Team'] == team]
    if not display_df.empty:
        print(display_df.to_string(index=False))
    
    # 팀별 개별 파일 저장 (필요 시 주석 해제)
    # display_df.to_csv(f'{team}_타격_요약_2020_2025.csv', index=False, encoding='utf-8-sig')

# 9. 전체 통합 요약본 저장
team_stats.to_csv('KBO_팀별_연도별_타격요약_통합.csv', index=False, encoding='utf-8-sig')

=== 10개 구단별 연도별 타격 요약 데이터 ===

[LG]
Team  Year   WAR  oWAR  wRC+_avg
  LG  2020 27.56 27.12    111.65
  LG  2021 22.81 21.54    101.32
  LG  2022 32.05 29.80    115.75
  LG  2023 33.67 32.99    118.56
  LG  2024 30.35 28.10    109.14
  LG  2025 36.33 34.92    118.67

[한화]
Team  Year   WAR  oWAR  wRC+_avg
  한화  2020  2.02  3.27     75.87
  한화  2021 13.49 11.72     88.32
  한화  2022 12.60 12.48     88.06
  한화  2023 14.01 13.80     89.36
  한화  2024 13.05 14.73     93.59
  한화  2025 20.00 19.19     99.39

[SSG]
Team  Year   WAR  oWAR  wRC+_avg
 SSG  2020  8.85  9.04     84.27
 SSG  2021 28.00 28.55    109.40
 SSG  2022 28.27 24.75    103.12
 SSG  2023 21.13 22.40    100.08
 SSG  2024 15.15 17.17     92.90
 SSG  2025 15.09 14.56     91.72

[삼성]
Team  Year   WAR  oWAR  wRC+_avg
  삼성  2020 13.92 13.23     86.72
  삼성  2021 18.14 19.34     97.99
  삼성  2022 17.06 19.27     97.97
  삼성  2023 16.37 16.64     94.31
  삼성  2024 20.08 16.74     95.88
  삼성  2025 29.63 26.98    108.26

[NC]
Team  Year   WA